# Task 2: Use SageMaker Feature Store

## Task 2.1: Feature Store options

There are three main ways to store features in Amazon SageMaker:
1. Using Amazon SageMaker Feature Store as an Amazon SageMaker Data Wrangler destination after preprocessing steps have been completed and features have been added.
2. Exporting a notebook from SageMaker Data Wrangler that runs through feature definition, feature group creation, and ingestion of data into SageMaker Feature Store.
3. Using the SageMaker Python SDK in a custom notebook that runs through feature definition, feature group creation, and ingestion of data into SageMaker Feature Store.

Each of these three options are outlined in following sections.

### Using SageMaker Feature Store as a SageMaker Data Wrangler destination

You can add SageMaker Feature Store as a destination in Amazon SageMaker Studio using the **Add destination** option. When you are finished with preprocessing steps in SageMaker Data Wrangler, you can choose **Add destination** in your flow. SageMaker Studio guides you through feature group creation and how to complete the necessary steps to ingest your preprocessed data into SageMaker Feature Store.

Refer to [Easily Create and Store Features in Amazon SageMaker Without Code](https://aws.amazon.com/blogs/machine-learning/easily-create-and-store-features-in-amazon-sagemaker-without-code/#save_features_to_feature_store) for more information about adding SageMaker Feature Store as a destination.

Refer to [Use Amazon SageMaker Feature Store with Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html) for more information about how to create a feature group in RSageMaker Studio.

### Exporting a notebook from SageMaker Data Wrangler

You can create feature groups using the **Export to** option in SageMaker Studio. When using the **Export to** option, you can create a notebook that includes all the commands necessary to create a feature group.

With a few customizations, you can run through the notebook to do the following:
- Create a feature definition based on your dataset.
- Create a feature group using the feature definition.
- Store the feature group in SageMaker Feature Store.
- Set up the inputs and outputs of a processing job.

This lab is similar to the exported notebook. This lab focuses on the first part of the exported notebook. It shows how to ingest data into the feature group and extract records from an online store and an offline store.

### Using the SageMaker Python SDK in a custom notebook

In this lab, you create a feature group and extract records from an online store and an offline store. You learn how SageMaker Feature Store works using a custom notebook. You set up the environment in Task 2. Then you complete the following tasks:

Task 2.3: Set up SageMaker Feature Store
- Create features in a notebook file.
- Create a feature group in SageMaker Feature Store.
- Confirm that the feature group has been created.
- View the feature group in SageMaker Studio.

Task 2.4: Query online and offline stores
- Ingest data into a feature group.
- Extract records from an online store.
- Extract records from an offline store using Amazon Athena.

## Task 2.2: Environment setup

Install packages and dependencies.

In [ ]:
#install-dependencies

import boto3
import json
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time
import uuid
import random
from sagemaker.session import Session
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup


region = boto3.Session().region_name
sess = boto3.Session(region_name=region)
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

Import the processed customer dataset.

In [ ]:
#explore-dataset

column_list = ['income','age','education','education_num','capital_gain','capital_loss','hours_per_week','sex','workclass','marital_status','occupation','relationship','race']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

Then, review a sample of the dataset in a table.

In [ ]:
#view-dataset

lab_test_data.dtypes
lab_test_data.head()

## Task 2.3: Set up SageMaker Feature Store

Create features to help train your model. Take the processed data from your Amazon SageMaker Data Wrangler lab to create a feature group using SageMaker Feature Store.

- Create features in a notebook file.
- Create a feature group in SageMaker Feature Store.
- Confirm the feature group has been created.
- View the feature group in SageMaker Studio.

### Task 2.3.1: Create features in a notebook file

To create a feature group, you need columns to assign for the **record_identifier_name** and the **event_time_feature_name**. To fulfill this requirement, add **record** and **event_time** columns to the dataset.
- The **record_identifier_name** refers to one of the names of a feature defined in the feature group's feature definitions. In this lab, you create a column of unique IDs called **record**.
- The **event_time_feature_name** is a point in time when a new event occurs that corresponds to the creation or update of a record in a feature group. All records in the feature group must have a corresponding event time. It can be used to track changes to a record over time. In this lab, you create a column called **event_time**.

Refer to [Get Started With Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-concepts.html) for more information about record identifier name or event time.

In [ ]:
#add-required-columns

# Add record and event_time columns
current_time_sec = int(round(time.time()))
lab_test_data.insert(0, 'record', range(0, 0 + len(lab_test_data)))
lab_test_data.insert(1, 'event_time', [current_time_sec]*len(lab_test_data))
lab_test_data['record'] = lab_test_data['record'].astype('string')
lab_test_data['event_time'] = lab_test_data['event_time'].astype('float64')

# Set the record-and-event_time-feature-names
record_identifier_feature_name = 'record'
event_time_feature_name = 'event_time'

# View the dataset
print(lab_test_data.dtypes)
lab_test_data.head()

You can add new features at any point during data processing, depending on your needs, either using the SageMaker Python SDK or SageMaker Studio.

In this lab, as part of your data preprocessing steps, add one feature using the SageMaker Python SDK. This feature is a weighted combination of two columns in the dataset and helps the model train more efficiently.

Create a workability feature, combining the age and hours_per_week columns to identify customers who are further along in their careers.

In [ ]:
#add-feature

lab_test_data = lab_test_data.assign(
    workability = 0.5*lab_test_data.age + 0.5*lab_test_data.hours_per_week)

lab_test_data.head()

### Task 2.3.2: Create a feature group in SageMaker Feature Store

To ingest features into SageMaker Feature Store, first define the feature definitions (feature name and data type) for all features that belong to the feature group.

A single feature corresponds to a column in your dataset. A feature group is a predefined schema for a collection of features. Each feature in the feature group has a specified data type and name. A single record in a feature group corresponds to a row in your dataframe. A feature store is a collection of feature groups. 

Refer to [Create, Store, and Share Features with Amazon SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) for more information about SageMaker Feature Store.

To start the process of making a feature definition, list out the schema for each feature.

In [ ]:
#list-column-schemas

column_schemas = [
    {
        "name": "record",
        "type": "string"
    },
    {
        "name": "event_time",
        "type": "float"
    },
    {
        "name": "income",
        "type": "string"
    },
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "education",
        "type": "float"
    },
    {
        "name": "education_num",
        "type": "float"
    },
    {
        "name": "capital_gain",
        "type": "long"
    },
    {
        "name": "capital_loss",
        "type": "long"
    },
    {
        "name": "hours_per_week",
        "type": "long"
    },
    {
        "name": "sex",
        "type": "float"
    },
    {
        "name": "workclass",
        "type": "array"
    },
    {
        "name": "marital_status",
        "type": "array"
    },
    {
        "name": "occupation",
        "type": "array"
    },
    {
        "name": "relationship",
        "type": "array"
    },
    {
        "name": "race",
        "type": "array"
    },
    {
        "name": "workability",
        "type": "float"
    }
]


Now that the schema is defined, create the input for the feature definitions. Set up a type mapping for the float and long dataset values of FRACTIONAL and INTEGRAL.

Then, create your feature definition, setting the values for **feature_name** and **feature_type** for all the columns in the schema that you defined.

In [ ]:
#create-feature-definitions

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

In SageMaker Feature Store, feature groups can be online-only, offline-only, or both. In this lab, you use both online and offline stores, so **enable_online_store** is set as **True**.
- Online store is primarily designed for supporting real-time predictions that need low millisecond latency reads and high throughput writes.
- Offline store is primarily intended for batch predictions and model training. Offline store is an append-only store and can be used to store and access historical feature data.

When your feature store is set to both online and offline store, any features ingested into online store are replicated in offline store. 

Configure the feature group, specifying options for the following settings:
- **feature_group_name**: The name of the feature group
- **feature_store_offline_s3_uri**: The Amazon Simple Storage Service (Amazon S3) bucket location to which SageMaker Feature Store writes data in the offline store of a feature group.
- **enable_online_store**: Controls whether or not an online store is enabled

In [ ]:
#configure-feature-store

# flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = "DataWranglerLab"
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

feature_store_offline_s3_uri = 's3://' + bucket
enable_online_store = True

Now that your feature store is configured, set an AWS Region and start a session. Then, set up a SageMaker client and a Feature Store runtime. Finally, set up the Feature Store session.

When setting up a feature store session, the **boto_session**, **sagemaker_client**, and **sagemaker_featurestore_runtime_client** values are defined.

In [ ]:
#set-up-sagemaker-feature-store-session

sagemaker_client = sess.client(service_name='sagemaker', region_name=region)
featurestore_runtime = sess.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=sess,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

Initialize the feature group using the parameters that you configured earlier and invoke the Feature Store API to create the feature group.

In [ ]:
#initialize-feature-group

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=enable_online_store
)

### Task 2.3.3: Confirm that the feature group has been created

Your feature group should be ready now. Confirm that the feature group has been created successfully.

Wait until the feature group is ready by using the Describe API. This function checks the response returned by the Describe API and waits for a status of **Created**.

In [ ]:
#wait-for-describe

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

List the feature groups using the ListFeatureGroups API.

In [ ]:
#list-feature-groups

response = sagemaker_client.list_feature_groups()
print(json.dumps(response, indent=4, sort_keys=True, default=str))

### Task 2.3.4: View the feature group in SageMaker Studio

You have created a SageMaker Feature Group using the SageMaker Python SDK. Now, review the feature group in SageMaker Studio to discover additional details.

1. In the navigation tab to the left of your notebook, choose the **SageMaker home** icon.

The next step opens a new tab in SageMaker Studio. To follow these directions, use one of the following options:
- **Option 1:** View the tabs side by side. To create a split screen view from the main SageMaker Studio window, either drag the **lab_5.ipynb** tab to the side or choose (right-click) the **lab_5.ipynb** tab and choose **New View for Notebook**. You can now have the directions visible as you explore the feature group.
- **Option 2:** Switch between the SageMaker Studio tabs to follow these instructions. When you are finished exploring the feature group, return to the notebook by selecting the **lab_5.ipynb** tab.

2. Expand the **Data** section, choose **Feature Store**.
3. The feature group you just created appears in the Feature Store tab. You can review details about the feature group. To locate more details, choose the feature group that starts with **FG-DataWranglerLab-**. Examine the following details while you explore SageMaker Feature Store in SageMaker Studio:
    - **Features**: Describes all the features in your feature group, including the **Type** and when the feature was created based on the **event_time** column.
    - **Details**: Outlines the metadata for the feature group, including the **Feature group status**, the **Record identifier** that you set earlier in the notebook, the **Store type** set to Online/Offline, and the **Table name** that you can use to query data from the offline feature store with Athena.
    - **Sample query**: Provides several sample queries that you can use to query data from the offline feature store.

## Task 2.4: Query online and offline stores

You have created a feature group. Now, ingest data into the feature group, extract records from an online store, and extract records from an offline store with Athena.

- Ingest data into a feature group.
- Extract records from an online store.
- Extract records from an offline store using Athena.

### Task 2.4.1: Ingest data into a feature group

After the feature group has been created, you put data into it. Use the PutRecord API call using **ingest()** with the SageMaker Python SDK. Whenever you create a feature group for the first time or want to add more records, you ingest the records into the feature group. 

For this dataset, ingestion takes 3–5 minutes. When the cell is complete when output like the following is displayed:

**IngestionManagerPandas(feature_group_name='FG-DataWranglerLab-13ee4f26', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fdb7fccee60>, sagemaker_session=<sagemaker.session.Session object at 0x7fdb82a900d0>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])**

In [ ]:
#ingest-records

feature_group.ingest(data_frame=lab_test_data, wait=True)

### Task 2.4.2: Extract records from an online store

The online store is particularly useful for inference tasks because you can return a subset of features quickly.

Now that your data is ingested, extract a record from an online store using **get_record**.

In [ ]:
#get-record

record = random.randint(0, len(lab_test_data.index)-1)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=str(record))

print(json.dumps(sample_record, indent=4, sort_keys=True, default=str))

Now, get several records from the feature group using **batch_get_record**. Several records have been chosen for you already, but feel free to change the records listed in **records_list**.

In [ ]:
#batch-get-record

records_list = ["7789", "5646", "309", "24528"]

batch_records = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": feature_group_name,
            "RecordIdentifiersValueAsString": records_list,
        }
    ]
)

print(json.dumps(batch_records, indent=4, sort_keys=True, default=str))

### Task 2.4.3 Extract records from an offline store using Athena

Now that you have extracted records from an online store, use Athena to extract records from an offline store. 

You can query the full dataset when you train and tune your model, or you can query a subset of records for inference. Because SageMaker Feature Store retains an event time for every record, you can train models with the exact set of features from a specific time in the past without the risk of including data from beyond that time. 

How would you adjust the query to change the subset of data returned from the offline store?

First, choose your query settings. You can customize your query to look at any subset of your data stored in the feature group. The following query is a basic SELECT query definition.

In [ ]:
#query-settings
# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup='primary',
        ConfigurationUpdates={
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

#Create the query
query = feature_group.athena_query()
table = query.table_name
query_string = f'SELECT * FROM "{table}" '
output_location = f's3://{bucket}/query_results/'

print(f'Athena query output location: \n{output_location}')

When your options are set, run the query, and display the results in the form of a table.

In [ ]:
#run-athena-query

query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

### Conclusion

Congratulations! You have used SageMaker Feature Store to make feature definitions for a feature group in SageMaker Studio and with the SageMaker Python SDK. With your newly created feature group, you are ready to train and tune your model using SageMaker Experiments in the next lab. Even later in the course, SageMaker Feature Store is useful for supplementing data for inference requests because of the low-latency GetRecord functionality using the online store. You continue working with this customer income dataset in the next lab.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.